In [11]:
import pyterrier as pt
import pandas as pd
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client
from feature_extraction import get_all_features
import pickle

ensure_pyterrier_is_loaded()
tira = Client()

In [27]:
DATASETS = {
    'longeval-train-20230513-training': 'train_2023', 
    'longeval-heldout-20230513-training': 'WT',
    'longeval-short-july-20230513-training': 'ST',
    'longeval-long-september-20230513-training': 'LT',
    'longeval-2023-01-20240423-training': 'train_2024',
    # 'longeval-2023-06-20240418-training': 'lag6',
    # 'longeval-2023-08-20240418-training': 'lag8',
}

In [30]:
for dataset_name, run_suffix in DATASETS.items():
    dataset = pt.get_dataset(f'irds:ir-benchmarks/{dataset_name}')

    bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', dataset)
    ltr = pt.io.read_results(f'../runs/ows_ltr/ows_ltr.{run_suffix}')

    print(dataset_name)
    print(pt.Experiment(
        [bm25, ltr],
        dataset.get_topics(variant='query'),
        dataset.get_qrels(variant='relevance'),
        eval_metrics=['ndcg_cut_10'],
        names=['BM25', 'BM25 + LMART']
    ).round(3))
    print()

longeval-train-20230513-training
           name  ndcg_cut_10
0          BM25        0.176
1  BM25 + LMART        0.282

longeval-heldout-20230513-training
           name  ndcg_cut_10
0          BM25        0.164
1  BM25 + LMART        0.205

longeval-short-july-20230513-training
           name  ndcg_cut_10
0          BM25        0.178
1  BM25 + LMART        0.240

longeval-long-september-20230513-training
           name  ndcg_cut_10
0          BM25        0.182
1  BM25 + LMART        0.242

longeval-2023-01-20240423-training
           name  ndcg_cut_10
0          BM25        0.199
1  BM25 + LMART        0.238

